In [56]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from time import time
import matplotlib.pyplot as plt
from operator import itemgetter

In [57]:
# Load training and test data into pandas dataframes
# ACCCESSING DATA FOLDER REFER THIS LINK 
# (https://stackoverflow.com/questions/7165749/open-file-in-a-relative-location-in-python)

import os

#TO ACCESS THE PATH IN WHICH CURRENT SCRIPT IS RUNNING
fileDir = os.path.dirname(os.path.realpath('__file__'))

#For accessing the file inside a sibling folder.
#data_type = 'raw' or 'processed'
#country   = 'A' or 'B' or 'C'
#person_type = 'hhold' or 'indiv'
#dataset = 'test' or 'train'

def return_file_name(data_type = 'raw', country = 'A', person_type = 'hhold', dataset = 'train'):
    path = '../data/' + data_type + '/' + country + '/' + country + '_' + person_type + '_' + dataset +'.csv'
    print("Building path for file : (" + path +")")
    filename = os.path.join(fileDir, path)
    filename = os.path.abspath(os.path.realpath(filename))
    #print(filename + "\n")
    return filename

def return_train_test(data_type = 'raw', country = 'A', person_type = 'hhold'):
    train = pd.read_csv(return_file_name(data_type = data_type, country = country, person_type = person_type, dataset = 'train'))
    test = pd.read_csv(return_file_name(data_type = data_type, country = country, person_type = person_type, dataset = 'test'))
    # merge training and test sets into one dataframe
    full = pd.concat([train, test])
    return train,test,full

In [58]:
# Get size of dataframes
train_set,test_set,full_set = return_train_test(data_type = 'raw', country = 'C', person_type = 'indiv')

#shape[0] -> # of rows
#shape[1] -> # of cols
print("\nTrain Shape : " + str(train_set.shape))
print("Test Shape : " + str(test_set.shape))
print("Full Shape : " + str(full_set.shape) + "\n")

Building path for file : (../data/raw/C/C_indiv_train.csv)
Building path for file : (../data/raw/C/C_indiv_test.csv)

Train Shape : (29913, 44)
Test Shape : (14701, 43)
Full Shape : (44614, 44)



In [59]:
#Looking for Nans
#return a formatted percentage from a fraction
def percentage(numerator, denomenator):
    
    if type(numerator) == pd.core.series.Series:
        return (numerator/denomenator*100)
    
    elif type(numerator) == int or type(numerator) == float:
        return '{:.1f}%'.format(float(numerator)/float(denomenator)*100) 
    
    else:
        print("check type")

In [60]:
#Get percentage by variable of values which are not NaN
def return_bad_cols(dataframe):
    fill_precent_counts = percentage(dataframe.count()-1, dataframe.shape[0]-1)
    bad_cols = fill_precent_counts[fill_precent_counts < 100]
    return bad_cols

train_bad_cols = return_bad_cols(train_set)
test_bad_cols = return_bad_cols(test_set)
test_bad_cols

Series([], dtype: float64)

In [61]:
#make directory for asving bad columns statistics
# refer link (https://stackoverflow.com/questions/273192/how-can-i-create-a-directory-if-it-does-not-exist)
stats_folder_path = '../data/stats/'

def make_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


def create_stats_folder():
    for name in ['A', 'B', 'C']:
        foldername = os.path.join(fileDir, stats_folder_path + name + '/individual')
        foldername = os.path.abspath(os.path.realpath(foldername))
        make_dir(foldername)
        foldername = os.path.join(fileDir, stats_folder_path + name + '/household')
        foldername = os.path.abspath(os.path.realpath(foldername))
        make_dir(foldername)
        print("Trying to create folder for " + name)
        
def return_bad_col_dtype(bad_columns, dataframe):
    data_type = []
    for name in bad_columns:
        data_type.append(dataframe[name].dtype)
    return data_type

create_stats_folder()



Trying to create folder for A
Trying to create folder for B
Trying to create folder for C


In [62]:
def return_unique_values(bad_columns, dataframe):
    unique_vals = []
    for name in bad_columns:
        vals = dataframe[dataframe[name].notnull()][name].unique()
        unique_vals.append(vals.tolist())
    return unique_vals

def return_max_val_repeated(bad_columns, dataframe):
    freq_count_num = []
    val_colon_freq_precent = []
    for name in bad_columns:
        #refer (https://www.w3resource.com/python-exercises/numpy/python-numpy-exercise-94.php)
        unique_elements, counts_elements = np.unique(dataframe[dataframe[name].notnull()][name], return_counts=True)
        val_with_max_freq = unique_elements[np.argmax(counts_elements)]
        max_freq = int(np.max(counts_elements) / np.sum(counts_elements) * 100)
        dict_f = {val_with_max_freq : max_freq}
    
        dict_temp = {}
        for val, count in zip(unique_elements, counts_elements):
            dict_temp[val] = int(count / np.sum(counts_elements) * 100)
        
        freq_count_num.append(dict_f)
        val_colon_freq_precent.append(dict_temp)
    return freq_count_num, val_colon_freq_precent



In [63]:
#############################
##    data set analysis    ##
#############################

def data_set_analysis(bad_columns, dataset):
    #make a data frame 
    #refer (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)
    bad_col_data = {'name': bad_columns.keys(), 'per_non_nan': bad_columns.values}
    bad_col_frame = pd.DataFrame(data=bad_col_data)
    
    #Add data type col
    bad_col_data_type = return_bad_col_dtype(bad_col_frame['name'], dataset)
    bad_col_frame['datatype'] = bad_col_data_type
    
    #unique vals col (not used)
    bad_col_unique_vals = return_unique_values(bad_col_frame['name'], dataset)
    #bad_col_frame['unique_vals'] = bad_col_unique_vals

    #show max freq count 
    bad_col_freq_count, temp = return_max_val_repeated(bad_col_frame['name'], dataset)
    bad_col_frame['freq_count'] = bad_col_freq_count
    bad_col_frame['unique_vals_colon_precent'] = temp

    return bad_col_frame
    
bad_frame_train = data_set_analysis(train_bad_cols, train_set)
bad_frame_test = data_set_analysis(test_bad_cols, test_set)
#make a data frame 
'''
#refer (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)
train_bad_col_data = {'name': train_bad_cols.keys(), 'per_non_nan': train_bad_cols.values}
train_bad_col_frame = pd.DataFrame(data=train_bad_col_data)

#Add data type col
bad_col_data_type = return_bad_col_dtype(train_bad_col_frame['name'], train_set)
train_bad_col_frame['datatype'] = bad_col_data_type

#unique vals col (not used)
bad_col_unique_vals = return_unique_values(train_bad_col_frame['name'], train_set)
#train_bad_col_frame['unique_vals'] = bad_col_unique_vals

#show max freq count 
bad_col_freq_count, temp = return_max_val_repeated(train_bad_col_frame['name'], train_set)
train_bad_col_frame['freq_count'] = bad_col_freq_count
train_bad_col_frame['unique_vals_colon_precent'] = temp
'''

#train_set[train_set.FGWqGkmD.isnull()].groupby('poor')['poor'].count()
#train[train.FGWqGkmD.notnull()].groupby('poor')['poor'].count()

#for col_name, precent in zip(train_bad_cols.keys(),train_bad_cols.values):
    #print(col_name + " " + str(precent))

"\n#refer (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)\ntrain_bad_col_data = {'name': train_bad_cols.keys(), 'per_non_nan': train_bad_cols.values}\ntrain_bad_col_frame = pd.DataFrame(data=train_bad_col_data)\n\n#Add data type col\nbad_col_data_type = return_bad_col_dtype(train_bad_col_frame['name'], train_set)\ntrain_bad_col_frame['datatype'] = bad_col_data_type\n\n#unique vals col (not used)\nbad_col_unique_vals = return_unique_values(train_bad_col_frame['name'], train_set)\n#train_bad_col_frame['unique_vals'] = bad_col_unique_vals\n\n#show max freq count \nbad_col_freq_count, temp = return_max_val_repeated(train_bad_col_frame['name'], train_set)\ntrain_bad_col_frame['freq_count'] = bad_col_freq_count\ntrain_bad_col_frame['unique_vals_colon_precent'] = temp\n"

In [64]:
#country = 'A'
#country = 'B'
country = 'C'

#type_people = 'household'
type_people = 'individual'

stats_name_train = os.path.join(fileDir, stats_folder_path + country + '/'+ type_people + '/' + country +'_' + type_people + '_train.csv')
stats_name_train = os.path.abspath(os.path.realpath(stats_name_train))

stats_name_test = os.path.join(fileDir, stats_folder_path + country + '/'+ type_people + '/' + country +'_' + type_people + '_test.csv')
stats_name_test = os.path.abspath(os.path.realpath(stats_name_test))


bad_frame_train.to_csv(stats_name_train, index=False)

bad_frame_test.to_csv(stats_name_test, index=False)